<a href="https://colab.research.google.com/github/tfreyd/TensorflowCoursera/blob/main/C4/W4/ungraded_labs/C4_W4_Lab_1_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install specific verison of tensorflow

#!pip install tensorflow==2.5.0

In [4]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt
print(tf.__version__)

2.6.0


In [10]:
def plot_series(time,series, format='-',start=0, end=None):
  plt.plot(time[start:end],series[start:end],format)
  plt.xlabel('Time')
  plt.ylabel('Value')
  plt.grid(True)

def trend(time, slope=0):
    return slope * time 
  
def seasonal_pattern(season_time):
    '''
    Just an aritrary patter, change it as you wish
    '''
    return np.where(season_time < 0.4, 
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp( 3 * season_time))
    
def seasonality(time, period, amplitude=1, phase=0):
      '''
      repeat the same pattern at each period
      '''
      season_time = ((time+phase) % period ) / period
      return amplitude * seasonal_pattern(season_time)

def noise(time, noise_level=1, seed= None):
  rnd=np.random.RandomState(seed)
  return rnd.randn(len(time))* noise_level


time=np.arange( 4 * 365 + 1, dtype='float32')
baseline=10
series=trend(time, 0.1)
baseline = 10 
amplitude = 40 
slope = 0.05 
noise_level = 5

# Create the series 
series = baseline + trend(time, slope) + seasonality ( time, period=365, amplitude=amplitude)

#Update with noise 
series += noise(time, noise_level, seed=42)

split_time = 1000
time_train= time[:split_time]
x_train =  series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

window_size = 20 
batch_size=20 
shuffle_bugger_size=1000


In [12]:
def windowed_dataset ( series, window_size, batch_size, shuffle_buffer):
  #expand to have 3 dimensions as a RNN cell takes input of 3 dimensions
  series = tf.expand_dims(series, axis=-1)
  #creation of the window dataset
  # slicing of the series to a tensor
  ds = tf.data.Dataset.from_tensor_slices(series)
  ds = ds.window(window_size + 1, shift = 1, drop_remainder=True)
  ds = ds.flat_map(lambda w: w.batch(window_size+1))
  ds = ds.shuffle(shuffle_buffer) #to avoid sequence bias
  ds = ds.map(lambda w: (w[:1], w[1:])) # split of the data in Xs and Y 
  return ds.batch(batch_size).refetch(1)